# nvImageCodecs batch processing examples

In [ ]:
import os
import os.path
from matplotlib import pyplot as plt
from nvidia import nvimgcodecs
resources_dir = "../../resources"


Load images

In [ ]:
image_paths = ["/base/4k_lossless.jp2", "/base/cat_q95_444.jpg",
               "/bmp/cat-111793_640.bmp"]
batch_size = len(image_paths)

data_list = []
for p in image_paths:
    with open(resources_dir + p, 'rb') as in_file:
        data = in_file.read()
        data_list.append(data)

Create decoder

In [ ]:
dec = nvimgcodecs.Decoder()

Decode

In [ ]:
image_list = dec.decode(data_list)

Show decoded images

In [ ]:
import cupy as cp

for img in image_list:
    print(img.__cuda_array_interface__)
    cp_img = cp.asarray(img)
    np_img = cp.asnumpy(cp_img)
    plt.figure()
    plt.imshow(np_img)


In [ ]:
enc = nvimgcodecs.Encoder()

In [ ]:
encoded_imgs = enc.encode(".jpg", image_list)

In [ ]:
i = 0
for enc_img in encoded_imgs:
    out_file_name = os.path.splitext(os.path.basename(image_paths[i]))[0]+ "_out.jpg"
    with open(out_file_name, "wb") as out_file:
        out_file.write(enc_img)
    i+=1

Decode all files in directory

In [ ]:
import glob

image_paths = glob.glob('{}/../build/bin/bin/in_mix/*.*'.format(resources_dir))
batch_size = 5
file_name_batches = [image_paths[i: i + batch_size] for i in range(0, len(image_paths), batch_size)] 
for file_name_batch in file_name_batches:
    data = []
    for path in file_name_batch:
        with open(path, "rb") as in_file:
            data.append(in_file.read())
            
    image_list = dec.decode(data)

    for img in image_list:
        cp_img = cp.asarray(img)
        np_img = cp.asnumpy(cp_img)
        plt.figure()
        plt.imshow(np_img)
